## Import Libraries and Initialize Objects

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np

# Assuming util and sort are custom modules you have access to
import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv

import csv
import numpy as np
from scipy.interpolate import interp1d

import ast
from tqdm import tqdm

import numpy as np
import pandas as pd

from IPython.display import display, Image
import PIL

/Users/syshin/Desktop/Syshin/Men-in-Black/Automatic License Plate Recognition/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Using CPU. Note: This module is much faster with a GPU.


### Initilize SORT tracker and Load Models

In [7]:
mot_tracker = Sort()
coco_model = YOLO('yolov8n.pt')
license_plate_detector = YOLO('models/11-08_best_weight_yolov8m_50_epoch.pt')

# Load the video
cap = cv2.VideoCapture('./sample.mp4')


## Process a Limited Number of Frames

In [8]:
import ipywidgets as widgets
from IPython.display import display

threshold_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Threshold:')
display(threshold_slider)

FloatSlider(value=0.5, description='Threshold:', max=1.0, step=0.01)

In [12]:
def show_image(img):
    _, encoded_image = cv2.imencode('.png', img)
    display(Image(data=encoded_image.tobytes()))

In [2]:
results = {}
vehicles = [1, 2, 3, 5, 7]  # Class IDs for vehicles

start_frame = 100
end_frame = 105
for frame_nmr in range(start_frame, end_frame):
    ret, frame = cap.read()
    if not ret:
        break  # Break the loop if no frame is returned

    results[frame_nmr] = {}
    # detect vehicles
    detections = coco_model(frame, device='mps')[0]
    detections_ = []
    for detection in detections.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = detection
        if int(class_id) in vehicles:
            detections_.append([x1, y1, x2, y2, score])

    # track vehicles
    track_ids = mot_tracker.update(np.asarray(detections_))

    # detect license plates
    license_plates = license_plate_detector(frame)[0]
    for license_plate in license_plates.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = license_plate

        # assign license plate to car
        xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)
        
        print(car_id)
        if car_id != -1:
            # crop license plate
            license_plate_crop = frame[int(y1):int(y2), int(x1):int(x2), :]

            # process license plate
            license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
            _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

            # read license plate number
            license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

            if license_plate_text is not None:
                results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                              'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                'text': license_plate_text,
                                                                'bbox_score': score,
                                                                'text_score': license_plate_text_score}}
    # Display the processed frame
    show_image(frame)
    # Optionally visualize the frame here using show_image(frame)


NameError: name 'cap' is not defined

In [5]:
# Close the video file
cap.release()

# Optionally, write results to CSV
write_csv(results, './test.csv')

## interpolate bounding boxes

In [6]:
def interpolate_bounding_boxes(data):
    # Extract necessary data columns from input data
    frame_numbers = np.array([int(row['frame_nmr']) for row in data])
    car_ids = np.array([int(float(row['car_id'])) for row in data])
    car_bboxes = np.array([list(map(float, row['car_bbox'][1:-1].split())) for row in data])
    license_plate_bboxes = np.array([list(map(float, row['license_plate_bbox'][1:-1].split())) for row in data])

    interpolated_data = []
    unique_car_ids = np.unique(car_ids)
    for car_id in unique_car_ids:

        frame_numbers_ = [p['frame_nmr'] for p in data if int(float(p['car_id'])) == int(float(car_id))]
        print(frame_numbers_, car_id)

        # Filter data for a specific car ID
        car_mask = car_ids == car_id
        car_frame_numbers = frame_numbers[car_mask]
        car_bboxes_interpolated = []
        license_plate_bboxes_interpolated = []

        first_frame_number = car_frame_numbers[0]
        last_frame_number = car_frame_numbers[-1]

        for i in range(len(car_bboxes[car_mask])):
            frame_number = car_frame_numbers[i]
            car_bbox = car_bboxes[car_mask][i]
            license_plate_bbox = license_plate_bboxes[car_mask][i]

            if i > 0:
                prev_frame_number = car_frame_numbers[i-1]
                prev_car_bbox = car_bboxes_interpolated[-1]
                prev_license_plate_bbox = license_plate_bboxes_interpolated[-1]

                if frame_number - prev_frame_number > 1:
                    # Interpolate missing frames' bounding boxes
                    frames_gap = frame_number - prev_frame_number
                    x = np.array([prev_frame_number, frame_number])
                    x_new = np.linspace(prev_frame_number, frame_number, num=frames_gap, endpoint=False)
                    interp_func = interp1d(x, np.vstack((prev_car_bbox, car_bbox)), axis=0, kind='linear')
                    interpolated_car_bboxes = interp_func(x_new)
                    interp_func = interp1d(x, np.vstack((prev_license_plate_bbox, license_plate_bbox)), axis=0, kind='linear')
                    interpolated_license_plate_bboxes = interp_func(x_new)

                    car_bboxes_interpolated.extend(interpolated_car_bboxes[1:])
                    license_plate_bboxes_interpolated.extend(interpolated_license_plate_bboxes[1:])

            car_bboxes_interpolated.append(car_bbox)
            license_plate_bboxes_interpolated.append(license_plate_bbox)

        for i in range(len(car_bboxes_interpolated)):
            frame_number = first_frame_number + i
            row = {}
            row['frame_nmr'] = str(frame_number)
            row['car_id'] = str(car_id)
            row['car_bbox'] = ' '.join(map(str, car_bboxes_interpolated[i]))
            row['license_plate_bbox'] = ' '.join(map(str, license_plate_bboxes_interpolated[i]))

            if str(frame_number) not in frame_numbers_:
                # Imputed row, set the following fields to '0'
                row['license_plate_bbox_score'] = '0'
                row['license_number'] = '0'
                row['license_number_score'] = '0'
            else:
                # Original row, retrieve values from the input data if available
                original_row = [p for p in data if int(p['frame_nmr']) == frame_number and int(float(p['car_id'])) == int(float(car_id))][0]
                row['license_plate_bbox_score'] = original_row['license_plate_bbox_score'] if 'license_plate_bbox_score' in original_row else '0'
                row['license_number'] = original_row['license_number'] if 'license_number' in original_row else '0'
                row['license_number_score'] = original_row['license_number_score'] if 'license_number_score' in original_row else '0'

            interpolated_data.append(row)

    return interpolated_data


In [7]:
# Load the CSV file
with open('test.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Interpolate missing data
interpolated_data = interpolate_bounding_boxes(data)

# Write updated data to a new CSV file
header = ['frame_nmr', 'car_id', 'car_bbox', 'license_plate_bbox', 'license_plate_bbox_score', 'license_number', 'license_number_score']
with open('test_interpolated.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(interpolated_data)


## Visualize

In [9]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=10, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv2.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv2.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv2.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv2.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv2.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv2.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv2.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img

In [14]:
results = pd.read_csv('./test_interpolated.csv')
video_path = 'sample.mp4'
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('sample_out.mp4', fourcc, fps, (width, height))
detection_threshold = 0.5  # Example threshold parameter

In [15]:
license_plate = {}
for car_id in np.unique(results['car_id']):
    max_ = np.amax(results[results['car_id'] == car_id]['license_number_score'])
    license_plate[car_id] = {'license_crop': None,
                             'license_plate_number': results[(results['car_id'] == car_id) &
                                                             (results['license_number_score'] == max_)]['license_number'].iloc[0]}
    cap.set(cv2.CAP_PROP_POS_FRAMES, results[(results['car_id'] == car_id) &
                                             (results['license_number_score'] == max_)]['frame_nmr'].iloc[0])
    ret, frame = cap.read()

    x1, y1, x2, y2 = ast.literal_eval(results[(results['car_id'] == car_id) &
                                              (results['license_number_score'] == max_)]['license_plate_bbox'].iloc[0].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

    license_crop = frame[int(y1):int(y2), int(x1):int(x2), :]
    license_crop = cv2.resize(license_crop, (int((x2 - x1) * 400 / (y2 - y1)), 400))

    license_plate[car_id]['license_crop'] = license_crop

In [16]:
frame_nmr = -1
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# read frames
ret = True
for frame_nmr in tqdm(range(total_frames), desc='Processing video frames'):
    ret, frame = cap.read()
    if not ret:
        break  # If no frame is returned, break out of the loop.

    df_ = results[results['frame_nmr'] == frame_nmr]
    for row_indx in range(len(df_)):
        # draw car
        car_x1, car_y1, car_x2, car_y2 = ast.literal_eval(df_.iloc[row_indx]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
        draw_border(frame, (int(car_x1), int(car_y1)), (int(car_x2), int(car_y2)), (0, 255, 0), 25,
                    line_length_x=200, line_length_y=200)

        # draw license plate
        x1, y1, x2, y2 = ast.literal_eval(df_.iloc[row_indx]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

        # crop license plate
        license_crop = license_plate[df_.iloc[row_indx]['car_id']]['license_crop']

        H, W, _ = license_crop.shape

        try:
            frame[int(car_y1) - H - 100:int(car_y1) - 100,
                  int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = license_crop

            frame[int(car_y1) - H - 400:int(car_y1) - H - 100,
                  int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = (255, 255, 255)

            (text_width, text_height), _ = cv2.getTextSize(
                license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                cv2.FONT_HERSHEY_SIMPLEX,
                4.3,
                17)

            cv2.putText(frame,
                        license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                        (int((car_x2 + car_x1 - text_width) / 2), int(car_y1 - H - 250 + (text_height / 2))),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        4.3,
                        (0, 0, 0),
                        17)

        except:
            pass

    out.write(frame)
    frame = cv2.resize(frame, (1280, 720))

    # cv2.imshow('frame', frame)
    # cv2.waitKey(0)


Processing video frames: 100%|████████████████████████████████████████| 3600/3600 [03:18<00:00, 18.17it/s]


In [17]:
out.release()
cap.release()

In [ ]:
pip install ipywidgets --upgrade

In [5]:
!jupyter labextension list

Config option `kernel_spec_manager_class` not recognized by `ListLabExtensionsApp`.
[W 2023-11-13 10:06:37.201 LabApp] Config option `kernel_spec_manager_class` not recognized by `LabApp`.
JupyterLab v3.4.2
/Users/syshin/miniforge3/share/jupyter/labextensions
        jupyterlab-jupytext v1.3.9 enabled OK (python, jupytext)
        jupyterlab_pygments v0.2.2 enabled OK (python, jupyterlab_pygments)
        jupyterlab-plotly v5.14.1 enabled OK
        @jupyter-widgets/jupyterlab-manager v3.1.0 enabled OK (python, jupyterlab_widgets)

Other labextensions (built into JupyterLab)
   app dir: /Users/syshin/miniforge3/share/jupyter/lab
        jupyterlab-dash v0.4.2 enabled OK


The following source extensions are overshadowed by older prebuilt extensions:
    @jupyter-widgets/jupyterlab-manager


In [4]:
!pip install jupyterlab_widgets

  Obtaining dependency information for jupyterlab_widgets from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata
  Using cached jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata (4.1 kB)
Using cached jupyterlab_widgets-3.0.9-py3-none-any.whl (214 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!jupyter labextension list

Config option `kernel_spec_manager_class` not recognized by `ListLabExtensionsApp`.
[W 2023-11-13 10:09:56.719 LabApp] Config option `kernel_spec_manager_class` not recognized by `LabApp`.
JupyterLab v3.4.2
/Users/syshin/miniforge3/share/jupyter/labextensions
        jupyterlab-jupytext v1.3.9 enabled OK (python, jupytext)
        jupyterlab_pygments v0.2.2 enabled OK (python, jupyterlab_pygments)
        jupyterlab-plotly v5.14.1 enabled OK
        @jupyter-widgets/jupyterlab-manager v5.0.9 enabled OK (python, jupyterlab_widgets)

Other labextensions (built into JupyterLab)
   app dir: /Users/syshin/miniforge3/share/jupyter/lab
        jupyterlab-dash v0.4.2 enabled OK



In [3]:
import sys
print(sys.version)

import matplotlib
print(matplotlib.get_backend())

print(matplotlib.rcsetup.all_backends)

3.9.6 (default, Aug 11 2023, 19:44:49) 
[Clang 15.0.0 (clang-1500.0.40.1)]
module://matplotlib_inline.backend_inline
['GTK3Agg', 'GTK3Cairo', 'GTK4Agg', 'GTK4Cairo', 'MacOSX', 'nbAgg', 'QtAgg', 'QtCairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']


In [5]:
!echo $DISPLAY

In [6]:
import tkinter
try:
    tkinter.Tk().withdraw()  # Open and then close a basic window
    print("GUI environment is available.")
except tkinter.TclError:
    print("GUI environment is not available.")


GUI environment is available.
